## Ads Performance

Write an pyspark code to find the ctr of each Ad.Round ctr to 2 decimal points. Order the result table by ctr in descending order and by ad_id in ascending order in case of a tie.
Ctr=Clicked/(Clicked+Viewed)

#### Difficult Level : EASY
##### DataFrame:
schema=StructType([
StructField('AD_ID',IntegerType(),True)
,StructField('USER_ID',IntegerType(),True)
,StructField('ACTION',StringType(),True)
])
# Define the data for the Ads table
data = [
(1, 1, 'Clicked'),
(2, 2, 'Clicked'),
(3, 3, 'Viewed'),
(5, 5, 'Ignored'),
(1, 7, 'Ignored'),
(2, 7, 'Viewed'),
(3, 5, 'Clicked'),
(1, 4, 'Viewed'),
(2, 11, 'Viewed'),
(1, 2, 'Clicked')
]

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StructField,StringType, StructType
import pandas as pd

In [4]:
spark=SparkSession.builder.master("local[*]").appName("AdsPerformance").getOrCreate()

In [8]:
schema=StructType([
StructField('AD_ID',IntegerType(),True)
,StructField('USER_ID',IntegerType(),True)
,StructField('ACTION',StringType(),True)
])
# Define the data for the Ads table
data = [
(1, 1, 'Clicked'),
(2, 2, 'Clicked'),
(3, 3, 'Viewed'),
(5, 5, 'Ignored'),
(1, 7, 'Ignored'),
(2, 7, 'Viewed'),
(3, 5, 'Clicked'),
(1, 4, 'Viewed'),
(2, 11, 'Viewed'),
(1, 2, 'Clicked')
]

df = spark.createDataFrame(data, schema)
df.show()

+-----+-------+-------+
|AD_ID|USER_ID| ACTION|
+-----+-------+-------+
|    1|      1|Clicked|
|    2|      2|Clicked|
|    3|      3| Viewed|
|    5|      5|Ignored|
|    1|      7|Ignored|
|    2|      7| Viewed|
|    3|      5|Clicked|
|    1|      4| Viewed|
|    2|     11| Viewed|
|    1|      2|Clicked|
+-----+-------+-------+



In [11]:
from pyspark.sql import functions as F
df_filter=df.groupBy('AD_ID').agg(F.sum(F.when(F.col('ACTION')=='Clicked',1).otherwise(0)).alias('Clicked'),F.sum(F.when(F.col('ACTION')=='Viewed',1).otherwise(0)).alias('Viewed'))
df_filter.show()

+-----+-------+------+
|AD_ID|Clicked|Viewed|
+-----+-------+------+
|    1|      2|     1|
|    2|      1|     2|
|    3|      1|     1|
|    5|      0|     0|
+-----+-------+------+



In [15]:
dfFinal=df_filter.withColumn('ctr',F.round(F.col('Clicked')/(F.col('Clicked')+F.col('Viewed')),2)).select('AD_ID','ctr')
dfFinal.show()

+-----+----+
|AD_ID| ctr|
+-----+----+
|    1|0.67|
|    2|0.33|
|    3| 0.5|
|    5|NULL|
+-----+----+



In [17]:
from pyspark.sql.window import Window
window_spec = Window.orderBy(F.col("ctr").desc(), F.col("ad_id").asc())
result_df = dfFinal.withColumn("rank", F.rank().over(window_spec)).select('AD_ID','ctr')
result_df.show()

+-----+----+
|AD_ID| ctr|
+-----+----+
|    1|0.67|
|    2|0.33|
|    3| 0.5|
|    5|NULL|
+-----+----+

